# Model training

In [1]:
! pip install numba
import pandas as pd # type: ignore
import numpy as np
import json
import time

## 1.Training with exist model
## - Data implementation
Run the following code to move all the graph to the folder for training and analysing.


In [3]:
! cp -r data4train/* DGB/data/ 

## - Model training and evaluation
Here we will train our graph with 4 models: *JODIE*, *DyRep*, *TGN*, *CAWN*.

The training command is callable in this file directly, but the testing command could only run in terminal.
Every time you train a graph,remember to rename the result in `DGB\log` by **MODEL+neg_sample+name of data+'_self_sup'** and move it into folder `DGB/log/neg_sample`

## JODIE

*JODIE* is a coupled recurrent neural network model that learns the embedding trajectories of users and items. JODIE employs two recurrent neural networks to update the embedding of a user and an item at every interaction.

![Image](img/JODIE.jpg)

run the following command to train dataset with jodie, replace the last value with name of alternative graph to train the respective data.

In [2]:
# training cmd
! sh train_jodie.sh vp14run1

INFO:root:Namespace(data='graph_data_vp14run1', bs=200, prefix='jodie_rnn', n_degree=10, n_head=2, n_epoch=50, n_layer=1, lr=0.0001, patience=5, n_runs=5, drop_out=0.1, gpu=0, node_dim=100, time_dim=100, backprop_every=1, use_memory=True, embedding_module='time', message_function='identity', memory_updater='rnn', aggregator='last', memory_update_at_end=False, message_dim=100, memory_dim=172, different_new_nodes=False, uniform=False, randomize_features=False, use_destination_embedding_in_message=False, use_source_embedding_in_message=False, dyrep=False, val_ratio=0.15, test_ratio=0.15)
The dataset has 15867 interactions, involving 14 different nodes
The training dataset has 9711 interactions, involving 12 different nodes
The validation dataset has 2380 interactions, involving 3 different nodes
The test dataset has 2380 interactions, involving 7 different nodes
The new node validation dataset has 0 interactions, involving 0 different nodes
The new node test dataset has 1823 interactions,

In [ ]:
# testing cmd
# NOTE: example of log file's name JODIE_hist_nre_graph_data_vp5run1_self_sup.log

! cd DGB
! python tgn/tgn_test_trained_model_self_sup.py -d graph_data_combined --use_memory --memory_updater rnn --embedding_module time --model jodie --neg_sample hist_nre --n_runs 5 --gpu 0 &> log/test/JODIE_combined.txt

## DyRep

![Image](img/Dyrep.jpg)

run the following command to train and test dataset with Dyrep, replace the last value with name of alternative graph to train the respective data.

In [ ]:
# training cmd
! sh train_dyrep.sh vp1run1

In [ ]:
# testing cmd
# NOTE: example of log file's name Dyrep_hist_nre_graph_data_vp1run1_self_sup.log

! cd DGB
! python tgn/tgn_test_trained_model_self_sup.py -d graph_data_vp1run1 --use_memory --memory_updater rnn --dyrep --use_destination_embedding_in_message --model dyrep --neg_sample hist_nre --n_runs 5 --gpu 0 &> log/test/Dyrep_vp1run1.txt

## TGN
Temporal Graph Networks (TGNs) operates on continuous-time dynamic graphs represented as a sequence of events,

![Image](img/TGN.jpg)

run the following command to train and test dataset with TGN, replace the last value with name of alternative graph to train the respective data.


In [ ]:
# training cmd
! sh train_tgn.sh vp1run2

In [ ]:
# testing cmd
# NOTE: example of log file's name TGN_hist_nre_graph_data_vp1run1_self_sup.log

! cd DGB
! python tgn/tgn_test_trained_model_self_sup.py -d graph_data_vp1run1 --use_memory --model tgn --neg_sample hist_nre --n_runs 5 --gpu 0 &> log/test/tgn_vp1run1.txt

## CAWN


![Image](img/CAWN.jpg)

run the following command to train and test dataset with Dyrep, replace the last value with name of alternative graph to train the respective data.

In [ ]:
# training cmd
! sh train_cawn.sh vp1run1

In [ ]:
# testing cmd
# NOTE: example of log file's name CAWN_hist_nre_graph_data_vp3run1_self_sup.log

! cd DGB
! python CAW/caw_test_trained_model_main.py -d vp1run1 --pos_dim 108 --bs 32 --n_degree 64 1 --mode i --bias 1e-5 --pos_enc lp --walk_pool sum --seed 0 --n_runs 5 --gpu 0 --neg_sample hist_nre &> log/test/CAWN_vp1run1.txt

## - Baseline calculation
Here we use **EdgeBank** as baseline.
run the following code to caculate Baseline of the predicted graph.

**!!** only the graphs trained before are available here.

In [ ]:
! cp -r DGB/data DGB/EdgeBank/data/ 
! python DGB/EdgeBank/link_pred/edge_bank_baseline.py --data graph_data_vp3run2 --mem_mode unlim_mem --n_runs 5 --neg_sample hist_nre


## 2. Training with our model
After comparing all the training results of the below models we would find that **JODIE** is one coming up with the best prediction. However, the model jodie still fail to predict the state of the predicted edge.
In my masterwork I would like to rewrite the embedding function and the loss function of **JODIE to make the state prediction possible.

- function from **JODIE**:

embedding function
$$ \mathbf{u(t)}=\sigma(W_1^u\mathbf{u(t^-)}+W_2^u\mathbf{i(t^-)}+W_3^uf+W^u_4 \Delta _u)$$
$$ \mathbf{i(t)}=\sigma(W_1^i\mathbf{i(t^-)}+W_2^i\mathbf{u(t^-)}+W_3^if+W^i_4 \Delta _i)$$

loss function(BCE)
$$L=-(j_{pos}\log{\tilde{j}}+j_{neg}log(1-\tilde{j}))$$

where

$$\tilde{j}(t+\Delta)=W_1\hat{u}(t+\delta)+W_2\bar{u}+W_3i(t+\Delta ^-)+W_4\bar{i}+B$$



- functions adapted in my work:

embedding function
$$ \mathbf{u(t)}=\sigma(W_1^u\mathbf{u(t^-)}+W_2^u\mathbf{i(t^-)}+W_3^uf+W^u_4s+W^u_5\Delta _u)$$
$$ \mathbf{i(t)}=\sigma(W_1^i\mathbf{i(t^-)}+W_2^i\mathbf{u(t^-)}+W_3^if+W^i_4s+W^u_5 \Delta _i)$$

we will change it from BCE to CE for predictiing state.

$$\tilde{j}(t+\Delta)=W_1\hat{u}(t+\delta)+W_2\bar{u}+W_3i(t+\Delta ^-)+W_4\bar{i}+W_5s+B$$

such changs are settled in `DGB/tgn/model/tgn.py` and `DGB/tgn/modules/embedding_module.py`

use the following command to start training.

In [ ]:
! sh train_jodie_state.sh vp1run1